# Mobile App for Lottery Addiction

## Introduction

In this project, we are going to contribute to the development of a mobile app that is meant to help lottery addicts better estimate their chances of winning.

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?


- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?


- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The [data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3665 drawings, dating from 1982 to 2018.

The scenario we're following throughout this project is fictional - the main purpose is to practice applying probability and combinatorics (permutations and combinations) concepts in a setting that simulates a real-world scenario.

## Core Functions

Our goal is to write code that can enable users to answer probability questions about playing the lottery. Throughout the project, we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

- A function that calculates factorials


- A function that calculates combinations

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done *without replacement*, which means once a number is drawn, it's not put back in the set.

In [1]:
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product


def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator/denominator

## One-Ticket Probability

Now, we focus on writing a function that calculates the probability of winning the big prize.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49


- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function


- The engineering team wants the function to print the probability value in a friendly way - in a way that people without any probability training are able to understand

In [19]:
def one_ticket_probability(user_numbers):    
    n_combinations = combinations(49, 6)
    probability_one_ticket = 1 / n_combinations
    print('''Your chances to win the big prize with the numbers {} are {:.7%}.
In other words, you have a 1 in {:,} chances to win.'''.format(user_numbers,
                                probability_one_ticket, int(n_combinations)))

In [21]:
one_ticket_probability([1, 2, 3, 4, 5])

Your chances to win the big prize with the numbers [1, 2, 3, 4, 5] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Historical Data Check for Canada Lottery

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

Next, we'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set has the following structure:

In [22]:
import pandas as pd

lottery_canada = pd.read_csv('649.csv')
lottery_canada.shape

(3665, 11)

In [26]:
lottery_canada.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


## Function for Historical Data Check

We're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team wants us to write a function that prints:
- the number of times the combination selected occurred in the Canada data set
- the probability of winning the big prize in the next drawing with that combination

We're going to begin by extracting all the winning numbers from the lottery data set. The `extract_numbers()` function will go over each row of the dataframe and extract the six winning numbers as a Python set.

In [27]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row


winning_numbers = lottery_canada.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

Below, we write the `check_historical_occurrence()` function that takes in the user numbers and the historical numbers and prints information with respect to the number of occurrences and the probability of winning in the next drawing.

In [35]:
def check_historical_occurrence(user_numbers, historical_numbers):   
    '''
    user_numbers: a Python list
    historical numbers: a pandas Series
    '''
    
    user_numbers_set = set(user_numbers)
    check_occurrence = historical_numbers == user_numbers_set
    n_occurrences = check_occurrence.sum()
    
    if n_occurrences == 0:
        print('''The combination {0} has never occured.
This doesn't mean it's more likely to occur now. 
Your chances to win the big prize in the next drawing using the combination {0} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers))
        
    else:
        print('''The number of times combination {0} has occured in the past is {1}.
This doesn't mean it's more likely to occur again. 
Your chances to win the big prize in the next drawing using the combination {0} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, n_occurrences))

In [36]:
test_input_2 = [33, 36, 37, 39, 8, 41]
check_historical_occurrence(test_input_2, winning_numbers)

The number of times combination [33, 36, 37, 39, 8, 41] has occured in the past is 1.
This doesn't mean it's more likely to occur again. 
Your chances to win the big prize in the next drawing using the combination [33, 36, 37, 39, 8, 41] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [37]:
test_input_3 = [1, 2, 3, 4, 5, 6]
check_historical_occurrence(test_input_3, winning_numbers)

The combination [1, 2, 3, 4, 5, 6] has never occured.
This doesn't mean it's more likely to occur now. 
Your chances to win the big prize in the next drawing using the combination [1, 2, 3, 4, 5, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


## Multi-Ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.



In [40]:
def multi_ticket_probability(n_tickets):
    n_combinations = combinations(49, 6)
    probability = n_tickets / n_combinations
    if n_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.6%}.
In other words, you have a 1 in {:,} chances to win.'''.format(probability, int(n_combinations)))
    else:
        combinations_simplified = round(n_combinations / n_tickets)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_tickets, probability,
                                                               combinations_simplified))

In [41]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------------') # output delimiter

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.000001%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.000007%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.000715%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 0.071511%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 0.500000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 different tic

## Less Winning Numbers - Function

Here, we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

The function below calculates the probability that a player's ticket matches exactly the given number of winning numbers (no more and no less). The function will not return the probability of having *at least* five winning numbers.

In [42]:
def probability_less_6(n_winning_numbers):
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_combinations_total = combinations(49, 6)    
    probability = successful_outcomes / n_combinations_total
    combinations_simplified = round(n_combinations_total/successful_outcomes)    
    print('''Your chances of having {} winning numbers with this ticket are {:.6%}.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability,
                                                               int(combinations_simplified)))

In [43]:
for test_input in [2, 3, 4, 5]:
    probability_less_6(test_input)
    print('--------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
--------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
--------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
--------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
--------------------------


## Conclusion

For the first version of the app, we managed to write four main functions for our app::

- `one_ticket_probability()` - calculates the probability of winning the big prize with a single ticket


- `check_historical_occurrence()` - checks whether a certain combination has occurred in the Canada lottery data set


- `multi_ticket_probability()` - calculates the probability for any number of of tickets between 1 and 13,983,816


- `probability_less_6()` - calculates the probability of having two, three, four or five winning numbers exactly